In [2]:
from collections import defaultdict, Counter
import pandas as pd
from itertools import islice

# Parsing amino acid FASTA files

In [3]:
def parse_fasta_amino_acid(file_path):
    """
    Parse a FASTA file to extract sequences along with their metadata.
    Returns a dictionary organized by protein type and virus type.
    """
    sequences = defaultdict(lambda: defaultdict(list))  # {virus_type: {protein_type: [sequences]}}
    with open(file_path, "r") as file:
        current_virus = None
        current_protein = None
        for line in file:
            line = line.strip()
            if line.startswith(">"):
                # Parse the header line
                header = line[1:]
                parts = header.split("|")
                protein_info = parts[1].split(" [")
                current_protein = protein_info[0].strip()
                current_virus = protein_info[1][:-1].strip()
            else:
                # Append sequence to the appropriate category
                sequences[current_virus][current_protein].append(line)
    return sequences

In [4]:
def compute_ngrams(sequence, n, is_amino_acid=True):
    """
    Compute all n-grams of length n from a given sequence,
    excluding n-grams with ambiguous characters.
    """
    if is_amino_acid:
        ambiguous_characters = {"B", "J", "O", "U", "X", "Z"}  # Ambiguous for amino acids
    else:
        ambiguous_characters = {"B", "D", "H", "K", "M", "R", "S", "V", "W", "Y", "N"}  # Ambiguous for nucleotides

    return [sequence[i:i+n] for i in range(len(sequence) - n + 1) 
            if not any(char in ambiguous_characters for char in sequence[i:i+n])]

In [5]:
def generate_ngram_matrix_amino_acid(fasta_path, n):
    """
    Generate an n-gram frequency matrix for clustering, organized by virus and protein type.
    Returns a pandas DataFrame with proteins as rows and n-grams as columns.
    """
    sequences = parse_fasta_amino_acid(fasta_path)
    ngram_counts = defaultdict(Counter)  # {protein_identifier: Counter of n-grams}
    protein_identifiers = []  # List of unique identifiers for clustering
    
    # Extract n-grams by protein and virus type
    for virus_type, proteins in sequences.items():
        for protein_type, seq_list in proteins.items():
            combined_sequence = "".join(seq_list)  # Combine all sequence fragments
            ngrams = compute_ngrams(combined_sequence, n, is_amino_acid=True)
            identifier = f"{virus_type} | {protein_type}"
            protein_identifiers.append(identifier)
            ngram_counts[identifier].update(ngrams)
    
    # Create a DataFrame for clustering
    all_ngrams = set(ngram for counter in ngram_counts.values() for ngram in counter)
    ngram_matrix = pd.DataFrame(index=protein_identifiers, columns=sorted(all_ngrams), dtype=int).fillna(0)
    
    for identifier, counter in ngram_counts.items():
        for ngram, count in counter.items():
            ngram_matrix.at[identifier, ngram] = count
            
    # Convert the DataFrame to integer type
    ngram_matrix = ngram_matrix.astype(int)

    # Reset the index to make 'virus_type' and 'protein_type' separate columns
    ngram_matrix.reset_index(inplace=True)
    ngram_matrix[['virus_type', 'protein_type']] = ngram_matrix['index'].str.split(' \| ', expand=True)
    ngram_matrix.drop(columns=['index'], inplace=True)
    columns_order = ['virus_type', 'protein_type'] + [col for col in ngram_matrix.columns if col not in ['virus_type', 'protein_type']]
    ngram_matrix = ngram_matrix[columns_order]

    return ngram_matrix

In [7]:
fasta_file = "data/ebola-amino-acid.fasta"
ns = [3, 5, 7]
for n in ns:
    ngram_matrix = generate_ngram_matrix_amino_acid(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_amino_acid_ebola.csv", index=False)
ngram_matrix

,virus_type,protein_type,AAAFHTF,AAAFHTY,AAAGLAW,AAAINDP,AAAPRAA,AAASAQR,AAATEAY,AAAVDAY,...,YYNPPHN,YYNVTCG,YYPTDVP,YYSTTIK,YYSTTIR,YYSTTSR,YYSVTCG,YYWTAVD,YYYLMSD,YYYMMTE
0,Bombali ebolavirus,nucleoprotein,0,0,0,0,0,7,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bombali ebolavirus,polymerase complex protein,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
2,Bombali ebolavirus,matrix protein,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
3,Bombali ebolavirus,spike glycoprotein,0,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bombali ebolavirus,small secreted glycoprotein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,"Ebola virus - Mayinga, Zaire, 1976",ssGP,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
280,"Ebola virus - Mayinga, Zaire, 1976",polymerase complex protein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
281,"Ebola virus - Mayinga, Zaire, 1976",VP24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
282,"Ebola virus - Mayinga, Zaire, 1976",polymerase,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [32]:
fasta_file = "data/mers-amino-acid.fasta"
ns = [3, 5, 7]
for n in ns:
    ngram_matrix = generate_ngram_matrix_amino_acid(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_amino_acid_mers.csv", index=False)
ngram_matrix

,virus_type,protein_type,AAAAKNK,AAACHLA,AAADGII,AAADLLV,AAAFSTL,AAAINAA,AAAINEA,AAAINKA,...,YYVKDGK,YYVKPGG,YYVNSVE,YYVQPGQ,YYVTFYV,YYYATFY,YYYATLY,YYYATVY,YYYDAQP,YYYVTFY
0,Betacoronavirus England 1,ORF8b,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Betacoronavirus England 1,nsp1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Betacoronavirus England 1,nsp2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Betacoronavirus England 1,nsp3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Betacoronavirus England 1,nsp4,0,2,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Human betacoronavirus 2c EMC/2012,ORF5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,Human betacoronavirus 2c EMC/2012,E protein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,Human betacoronavirus 2c EMC/2012,M protein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208,Human betacoronavirus 2c EMC/2012,N protein,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
fasta_file = "data/marburg-amino-acid.fasta"
ns = [3, 5, 7]
for n in ns:
    ngram_matrix = generate_ngram_matrix_amino_acid(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_amino_acid_marburg.csv", index=False)
ngram_matrix

,virus_type,protein_type,AAAFDAY,AADIFSR,AADKMLK,AADKMSK,AAEIENN,AAFDAYL,AAFHSML,AAFMYGN,...,YYDELCC,YYDELCD,YYDELCS,YYDELHD,YYDELQS,YYDELRS,YYGQVQL,YYNDILK,YYNSDKD,YYWGMLL
0,Orthomarburgvirus marburgense,nucleoprotein,0,0,0,0,62,0,0,0,...,0,0,0,0,0,0,0,0,62,0
1,Orthomarburgvirus marburgense,polymerase complex protein,3,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Orthomarburgvirus marburgense,matrix protein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Orthomarburgvirus marburgense,glycoprotein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Orthomarburgvirus marburgense,minor nucleoprotein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"Marburg virus - Musoke, Kenya, 1980",VP40,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,"Marburg virus - Musoke, Kenya, 1980",glycoprotein,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,"Marburg virus - Musoke, Kenya, 1980",VP30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,"Marburg virus - Musoke, Kenya, 1980",VP24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Parsing nucleotide FASTA files


In [24]:
def parse_fasta_nucleotide(file_path):
    """
    Function to parse a FASTA file and extract virus types and sequences
    """
    virus_sequences = defaultdict(str)
    
    with open(file_path, 'r') as file:
        current_virus = None
        for line in file:
            if line.startswith('>'):
                header = line.strip()
                virus_index = header.find("virus")
                if virus_index != -1:
                    current_virus = header[:virus_index + len("virus")].replace(">", "").strip()
                    if '|' in current_virus:
                        current_virus = current_virus.split('|')[1].strip()
                    if "UNVERIFIED:" in current_virus:
                        current_virus = current_virus.split('UNVERIFIED:')[1].strip()
                else:
                    current_virus = "Unknown Virus"
            else:
                virus_sequences[current_virus] += line.strip()
    
    return virus_sequences

In [25]:
def generate_ngram_matrix_nucleotide(fasta_path, n):
    """
    Generate an n-gram frequency matrix for clustering, organized by virus and protein type.
    Returns a pandas DataFrame with proteins as rows and n-grams as columns.
    """
    all_ngrams = set()
    ngram_counts = {}

    virus_sequences = parse_fasta_nucleotide(fasta_file)

    # Count n-grams for each virus type
    for virus, sequence in virus_sequences.items():
        ngrams = compute_ngrams(sequence, n, is_amino_acid=False)
        ngram_counts[virus] = Counter(ngrams)
        all_ngrams.update(ngrams)

    # Create a DataFrame
    ngram_matrix = pd.DataFrame(index=ngram_counts.keys(), columns=sorted(all_ngrams), dtype=int).fillna(0)

    for virus, counts in ngram_counts.items():
        for ngram, count in counts.items():
            ngram_matrix.at[virus, ngram] = count

    # Convert float values to int
    ngram_matrix = ngram_matrix.astype(int)
    
    return ngram_matrix

In [34]:
fasta_file = "data/ebola-nucleotide.fasta"
ns = [3, 5, 7] 
for n in ns:
    ngram_matrix = generate_ngram_matrix_nucleotide(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_nucleotide_ebola.csv")
ngram_matrix

,AAAAAAA,AAAAAAC,AAAAAAG,AAAAAAT,AAAAACA,AAAAACC,AAAAACG,AAAAACT,AAAAAGA,AAAAAGC,...,TTTTTCG,TTTTTCT,TTTTTGA,TTTTTGC,TTTTTGG,TTTTTGT,TTTTTTA,TTTTTTC,TTTTTTG,TTTTTTT
Bombali ebolavirus,12,16,6,42,31,34,7,19,61,1,...,0,8,17,17,8,11,24,14,31,17
Bundibugyo ebolavirus,25,69,4,116,93,36,0,24,87,24,...,8,12,33,24,12,70,72,4,51,14
Tai Forest ebolavirus,7,19,3,15,27,12,3,9,26,9,...,3,0,15,0,6,5,9,12,11,6
Sudan ebolavirus,62,58,10,178,203,27,12,0,154,56,...,0,68,72,12,23,48,43,35,39,10
Reston ebolavirus,49,144,82,95,111,73,28,105,140,7,...,0,94,58,25,47,31,89,79,35,21
Zaire ebolavirus,957,3285,2023,2657,3284,2243,58,3270,3398,789,...,143,1460,1854,970,928,1104,908,607,1600,139
Mutant Zaire ebolavirus,26,87,36,81,83,51,11,88,100,15,...,13,47,46,35,27,37,18,35,40,12
Mutant Bombali ebolavirus,1,2,1,5,3,5,1,2,8,0,...,0,1,2,2,1,2,3,2,4,2
Ebola virus,216,771,234,556,756,448,94,769,772,17,...,107,418,537,310,216,140,216,217,337,107
Bundibugyo virus,2,6,0,10,7,3,0,2,7,2,...,1,1,2,2,1,6,6,0,4,1


In [27]:
print(ngram_matrix.iloc[:, 0])

Bombali ebolavirus             407
Bundibugyo ebolavirus          848
Tai Forest ebolavirus          217
Sudan ebolavirus              1433
Reston ebolavirus             1401
Zaire ebolavirus             33534
Mutant Zaire ebolavirus        874
Mutant Bombali ebolavirus       50
Ebola virus                   6952
Bundibugyo virus                70
Cote d'Ivoire ebolavirus        72
Zaire Ebola virus              217
Reston Ebola virus              66
Name: AAAAA, dtype: int64


In [35]:
fasta_file = "data/marburg-nucleotide.fasta"
ns = [3, 5, 7]
for n in ns:
    ngram_matrix = generate_ngram_matrix_nucleotide(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_nucleotide_marburg.csv")
ngram_matrix

,AAAAAAA,AAAAAAC,AAAAAAG,AAAAAAT,AAAAACA,AAAAACC,AAAAACG,AAAAACT,AAAAAGA,AAAAAGC,...,TTTTTCG,TTTTTCT,TTTTTGA,TTTTTGC,TTTTTGG,TTTTTGT,TTTTTTA,TTTTTTC,TTTTTTG,TTTTTTT
Marburg marburgvirus,1,260,94,129,315,108,43,481,217,177,...,72,245,150,41,201,31,164,92,95,0
Orthomarburgvirus,7,74,43,29,54,57,29,127,68,70,...,15,85,104,23,57,34,57,37,54,5
Mutant Orthomarburgvirus,0,7,2,4,9,2,1,14,5,4,...,2,6,3,1,5,1,5,1,4,0
Mutant Marburg marburgvirus,0,21,6,12,27,6,3,42,15,12,...,6,18,9,3,15,3,15,3,12,0
Lake Victoria marburgvirus,2,135,63,98,171,106,42,261,110,113,...,30,160,117,48,119,43,106,92,71,0
Marburg virus,1,12,6,9,14,7,1,20,9,5,...,1,10,6,2,9,4,6,6,6,0


In [37]:
fasta_file = "data/mers-nucleotide.fasta"
ns = [3, 5, 7]
for n in ns:
    ngram_matrix = generate_ngram_matrix_nucleotide(fasta_file, n)
    ngram_matrix.to_csv(f"csv_data/{n}gram_matrix_nucleotide_mers.csv")
ngram_matrix

,AAAAAAA,AAAAAAC,AAAAAAG,AAAAAAT,AAAAACA,AAAAACC,AAAAACG,AAAAACT,AAAAAGA,AAAAAGC,...,TTTTTCG,TTTTTCT,TTTTTGA,TTTTTGC,TTTTTGG,TTTTTGT,TTTTTTA,TTTTTTC,TTTTTTG,TTTTTTT
Betacoronavirus,15,0,2,1,4,2,2,10,2,6,...,0,8,12,4,2,6,4,4,0,0
Middle East respiratory syndrome-related coronavirus,3376,8,1003,161,860,447,454,2225,948,1373,...,8,1874,2729,923,729,1300,1098,887,312,41
MAG: Middle East respiratory syndrome-related coronavirus,5,1,5,1,4,1,2,8,5,8,...,0,13,11,11,5,10,7,7,0,2
Mutant Middle East respiratory syndrome-related coronavirus,24,0,2,1,2,1,1,5,2,3,...,0,4,7,2,2,3,3,2,2,0
Middle East respiratory syndrome coronavirus,1004,6,395,21,382,192,194,962,385,575,...,2,771,1196,388,264,576,490,385,117,1
Unknown Virus,5,0,0,0,1,0,1,2,0,6,...,1,5,4,2,1,5,2,1,1,0
Human betacoronavirus,16,0,5,1,8,4,4,20,5,12,...,0,16,24,9,4,11,8,8,4,0
Coronavirus,5,0,0,0,4,1,0,2,2,2,...,1,5,7,5,2,5,1,2,0,0
